In [3]:
# -*- coding: utf-8 -*-
import requests
import sys
import time
import pandas as pd
import json
from bs4 import BeautifulSoup
import multiprocessing

HTML_PARSER   = "html.parser"
USER_AGENT    = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/59.0.3071.115 Safari/537.36'
HEADERS       = {'User-Agent': USER_AGENT}
MOVIE_ID_URL  = 'https://www.imdb.com/title/'
KEY = '<e7a9762dabfdc186e180a0791bb06e7c>'
head = ["id", "rating", "dircetor",
			"stars", "style", "runtime", "reviews_user", "reviews_critic", "budget"]
fileName = 'v7_auto_labeled_sort_with_box_office_1997-2017.csv'

def Movie(title):
	data_list = []
	offic_metadata_PATTERN = 'https://api.themoviedb.org/3/movie/'+ title +'?api_key=e7a9762dabfdc186e180a0791bb06e7c'
	offic_results = requests.get(offic_metadata_PATTERN.format(key=KEY))
	imdb_api_response = offic_results.json()
	runtime = imdb_api_response['runtime']
	budget  = imdb_api_response['budget']
	movie_req = requests.get(MOVIE_ID_URL+title, headers = HEADERS)
	if movie_req.status_code == requests.codes.ok:
		print('URL OK')
		stars = ''
		soup = BeautifulSoup(movie_req.content, HTML_PARSER)
		if len(soup.select('span[itemprop="ratingValue"]')) == 0:
			rank = 'no data'
		else:
			rank = soup.select('span[itemprop="ratingValue"]')[0].text
		if len(soup.select('span[itemprop="name"]')) == 0:
			dircetor = 'no data'
		else:
			dircetor = soup.select('span[itemprop="name"]')[0].text
		if len(soup.select('span[itemprop="name"]')) == 0:
			stars = 'no data'
		else:
			for str in range(0,len(soup.select('span[itemprop="actors"]'))):
				stars+=soup.select('span[itemprop="actors"]')[str].text.strip()
		if len(soup.select('span[itemprop="genre"]')) == 0:
			style = 'no data'
		else:
			style = soup.select('span[itemprop="genre"]')[0].text.strip()
		if len(soup.select('div.titleReviewbarItemBorder > div > span > a')) == 0:
			reviews_user = 'no data'
			reviews_critic = 'no data'
		elif len(soup.select('div.titleReviewbarItemBorder > div > span > a')) == 1:
			if 'user' in soup.select('div.titleReviewbarItemBorder > div > span > a')[0].text.strip():
				reviews_user = soup.select('div.titleReviewbarItemBorder > div > span > a')[0].text.strip().replace(' user','')
				reviews_critic = 'no data'
			if 'critic' in soup.select('div.titleReviewbarItemBorder > div > span > a')[0].text.strip():
				reviews_user = 'no data'
				reviews_critic = soup.select('div.titleReviewbarItemBorder > div > span > a')[0].text.strip().replace(' critic','')
		elif len(soup.select('div.titleReviewbarItemBorder > div > span > a')) == 2:
			if 'user' in soup.select('div.titleReviewbarItemBorder > div > span > a')[0].text.strip():
				reviews_user = soup.select('div.titleReviewbarItemBorder > div > span > a')[0].text.strip().replace(' user','')
			if 'critic' in soup.select('div.titleReviewbarItemBorder > div > span > a')[1].text.strip():
				reviews_critic = soup.select('div.titleReviewbarItemBorder > div > span > a')[1].text.strip().replace(' critic','')
	data_list.append([[title, rank, dircetor, stars, style, runtime, reviews_user, reviews_critic, budget]])
	return data_list


def list2csv(result):
	for items in result:
		for data_list in items.get():
			print(data_list)
			df = pd.DataFrame(data_list , columns = head)
			df.to_csv("movie2.csv", mode='a', header=False)

def movielist():
	pool = multiprocessing.Pool(processes=7)
	print('cpu_count:',multiprocessing.cpu_count())
	result = []
	df = pd.read_csv(fileName)
	for x in range(0,len(df.imdb_id)):
		if x%7 == 0:
			result.append(pool.apply_async(Movie,(df.imdb_id[0+x],)))
			result.append(pool.apply_async(Movie,(df.imdb_id[1+x],)))
			result.append(pool.apply_async(Movie,(df.imdb_id[2+x],)))
			result.append(pool.apply_async(Movie,(df.imdb_id[3+x],)))
			result.append(pool.apply_async(Movie,(df.imdb_id[4+x],)))
			result.append(pool.apply_async(Movie,(df.imdb_id[5+x],)))
			result.append(pool.apply_async(Movie,(df.imdb_id[6+x],)))
	list2csv(result)
	pool.close()
	pool.join()

if __name__ == '__main__':
	start = time.time()
	movielist()
	print('All cost',time.time()-start)

cpu_count: 4
URL OK
URL OK
URL OK
URL OK
URL OK
URL OK
URL OK
[['tt0120376', '5.7', 'George Huang', 'Will Friedle,Jennifer Love Hewitt,Marley Shelton', 'Comedy', 85, '43', '3', 15000000]]
[['tt0118870', '4.6', 'John Ridley', 'David Caruso,Kelly Lynch,Stacey Dash', 'Crime', 96, '13', '6', 0]]
[['tt0130297', '7.1', 'Joe Gayton', 'Samantha Mathis,Barbara Pilavin,Kimberly Scott', 'Drama', None, '10', '1', 0]]
[['tt0028905', '7.1', 'Pierre Chenal', 'Pierre Blanchar,Isa Miranda,Catherine Fonteney', 'Drama', 83, '2', '1', 0]]
[['tt0119327', '5.2', 'Robert Downey Sr.', 'Alyssa Milano,Mark Boone Junior,Malcolm McDowell', 'Comedy', 92, '33', '13', 0]]
[['tt0119401', '5.3', 'Mike Barker', 'John Hannah,Helen McCrory,Jason Flemyng', 'Comedy', 95, '3', '1', 0]]
[['tt0118811', '6.5', 'Peter Svatek', 'Rutger Hauer,Bronwen Booth,Charles Edwin Powell', 'Adventure', 91, '8', '2', 0]]
URL OK
URL OK
URL OK
URL OK
URL OK
URL OK
[['tt7253216', 'no data', 'Alex Lutz', 'Alex Lutz,Tom Dingler,Pascale Arbillot',